In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from PyPDF2 import PdfReader
from langchain.chat_models import ChatOpenAI
from langchain.chains import QAGenerationChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import json

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "xxx"

In [ ]:
reader = PdfReader('/kaggle/input/track2-aidea/track2-问答式科研知识库/141.pdf')

In [ ]:
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
text_splitter = CharacterTextSplitter(        
    separator = ".",
    chunk_size = 300,
    chunk_overlap  = 200,
    length_function = len,
)


In [ ]:
texts = text_splitter.split_text(raw_text)

In [ ]:
chain = QAGenerationChain.from_llm(ChatOpenAI(temperature=0))
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
paper = []
for n in texts:
    try:
        qa = chain.run(n)
        query = qa[0]["question"]
        docs = docsearch.similarity_search(query)
        data = {
            "input": docs[0].page_content,
            "question": qa[0]["question"],
            "ans": qa[0]["answer"]
        }
        paper.append(data)
    except json.JSONDecodeError:
        # 处理无效控制字符的情况
        print("Skipping documents or answers that contain invalid control characters")
    
    print(data)
    print()    
print("------------")
print(paper)

In [ ]:
json_str=json.dumps(paper, ensure_ascii=False)

with open('141.json', 'w') as f:
    json.dump(json_str, f)

